In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)


In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [6]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

bluff nz
lixourion gr
new norfolk au
taoudenni ml
sayyan ye
mataura pf
anamur tr
fairbanks us
iqaluit ca
temaraia ki
portland au
mataura pf
salalah om
rikitea pf
cherskiy ru
nautanwa in
tuktoyaktuk ca
new norfolk au
esperance au
airai pw
provideniya ru
hermanus za
port elizabeth za
kuala sungai baru my
cape town za
mataura pf
tuktoyaktuk ca
carnarvon au
askarovo ru
bluff nz
mataura pf
castro cl
blyth gb
el wasta eg
hobart au
saint-philippe re
udachnyy ru
busselton au
barrow us
cururupu br
araouane ml
ushuaia ar
ushuaia ar
tiksi ru
rikitea pf
price us
bambous virieux mu
tasiilaq gl
albany au
hilo us
mataura pf
sabang id
belushya guba ru
arraial do cabo br
new norfolk au
tazirkah tn
dikson ru
upernavik gl
kodiak us
san patricio mx
kapaa us
baruun-urt mn
marshall us
monte alegre br
changzhou cn
saint-philippe re
dundee za
sao joao da barra br
carnarvon au
yerbogachen ru
cherskiy ru
puerto ayora ec
bielawa pl
praia cv
mataura pf
carnarvon au
arraial do cabo br
northam au
qaanaaq gl
sola vu

ushuaia ar
new norfolk au
rikitea pf
longlac ca
airai pw
khatanga ru
busselton au
galesong id
byron bay au
cabo san lucas mx
avarua ck
monte carmelo br
punta arenas cl
bluff nz
alofi nu
carnarvon au
sitka us
derbent ru
airai pw
mys shmidta ru
rikitea pf
jamestown sh
ushuaia ar
novoagansk ru
rikitea pf
nanortalik gl
cape town za
inongo cd
kharan pk
morant bay jm
avarua ck
kapaa us
brae gb
avarua ck
backnang de
georgetown gy
puerto escondido mx
yellowknife ca
atuona pf
atuona pf
vaini to
bluff nz
jaru br
hermanus za
avarua ck
san patricio mx
tuktoyaktuk ca
new norfolk au
cidreira br
avarua ck
port blair in
sentyabrskiy ru
butaritari ki
ushuaia ar
kulhudhuffushi mv
kruisfontein za
atuona pf
nolinsk ru
tumannyy ru
bluff nz
ventspils lv
kazalinsk kz
bredasdorp za
saint-philippe re
nizhneyansk ru
busselton au
busselton au
butaritari ki
carnarvon au
butaritari ki
dakar sn
busselton au
hobart au
butaritari ki
dejen et
sinnamary gf
new norfolk au
abaete br
lebu cl
ha noi vn
andenes no
hobart au

ushuaia ar
tiksi ru
faya td
karaidel ru
longyan cn
lavumisa sz
punta arenas cl
cascais pt
rikitea pf
saskylakh ru
jamestown sh
vostok ru
columbus us
atuona pf
atuona pf
sola vu
jamestown sh
muli mv
ushuaia ar
yar-sale ru
kaeo nz
korla cn
bluff nz
bluff nz
nsukka ng
yellowknife ca
ushuaia ar
zhangjiakou cn
tasiilaq gl
tiksi ru
khatanga ru
aykhal ru
chuy uy
touros br
upernavik gl
castro cl
busselton au
lethem gy
atuona pf
souillac mu
el viso del alcor es
pechenga ru
dikson ru
new norfolk au


In [11]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

602